# *Graph Rag: Fake News Detection*
### *Aman Pawar*

In [ ]:
# -*- coding: utf-8 -*-
"""
Graph-RAG Implementation using Similarity-Based Neighborhood Exploration.

This script implements a Retrieval-Augmented Generation (RAG) approach
for text classification (real vs. fake news tweets). It enhances the standard
RAG by incorporating a graph-like retrieval step. Instead of just retrieving
the top-k similar documents, it retrieves the top-k documents and then
retrieves the neighbors of those documents in the embedding space, simulating
a 1-hop graph traversal based on semantic similarity.

Key Components:
1. Configuration: Sets up hyperparameters and paths. Includes DEBUG_DATALOADER flag.
2. Data Loading: Loads and preprocesses data from Excel files.
3. Custom Dataset (GraphRagNewsDataset): Defined globally. Integrates retrieval.
4. Retriever Setup: Uses SentenceTransformer and FAISS.
5. Graph-like Retrieval Function: Finds primary and secondary neighbors.
6. Classifier Model: Uses a transformer model (e.g., BERT).
7. Training Loop: Fine-tunes the classifier.
8. Evaluation: Measures performance.
"""

import pandas as pd
import numpy as np
import torch
import torch.multiprocessing as mp # Import torch multiprocessing
from torch.utils.data import Dataset, DataLoader
from torch.optim import AdamW
import torch.nn as nn
from transformers import AutoTokenizer, AutoModelForSequenceClassification, get_linear_schedule_with_warmup
from sentence_transformers import SentenceTransformer
import faiss # For efficient similarity search
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from tqdm.auto import tqdm
import os
import time
import gc # Garbage collector
import traceback # For detailed error printing
import logging # Using logging for better output management

# --- Set Environment Variable to Suppress Tokenizer Parallelism Warning ---
os.environ["TOKENIZERS_PARALLELISM"] = "false"

# --- Configure Logging ---
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - [%(processName)s] %(message)s') # Added process name


# --- 1. Configuration ---
class Config:
    """Configuration class for hyperparameters and settings."""
    # File paths (Update these paths if your files are located elsewhere)
    # IMPORTANT: Ensure these paths are correct relative to where you run the script.
    train_file = '../Constraint_English_Train.xlsx'
    val_file = '../Constraint_English_Val.xlsx'
    test_file = '../english_test_with_labels.xlsx'

    # Model names
    retriever_model_name = 'all-MiniLM-L6-v2'
    classifier_model_name = 'bert-base-uncased'

    # RAG parameters
    num_retrieved_docs_primary = 3
    num_retrieved_docs_secondary = 2
    max_total_retrieved_docs = 5 # Total unique docs (primary + secondary)

    # Training parameters
    max_seq_length = 512
    batch_size = 8
    epochs = 3
    learning_rate = 2e-5
    warmup_steps = 100
    gradient_accumulation_steps = 1

    # Hardware and reproducibility
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    seed = 42

    # Output directory
    output_dir = "./graph_rag_classifier_model"

    # --- DEBUGGING FLAG ---
    # >>> SET THIS TO True <<< to debug DataLoader worker errors by forcing num_workers=0
    # >>> SET THIS TO False <<< for potentially faster training with multiple workers (if errors are resolved)
    DEBUG_DATALOADER = True # <--- CHANGE THIS TO True TO GET DETAILED ERROR TRACEBACKS

# --- 2. Data Loading Function ---
def load_data(filepath):
    """Loads data from Excel, cleans, maps labels."""
    if not os.path.exists(filepath):
        logging.error(f"Data file not found: {filepath}")
        return None
    try:
        df = pd.read_excel(filepath)
        if 'tweet' not in df.columns or 'label' not in df.columns:
            logging.error(f"Excel file {filepath} must contain 'tweet' and 'label' columns.")
            return None

        initial_count = len(df)
        df = df.dropna(subset=['tweet'])
        df['tweet'] = df['tweet'].astype(str)
        dropped_nan_tweet = initial_count - len(df)

        df['label'] = df['label'].map({'real': 1, 'fake': 0})
        initial_count_before_label_drop = len(df)
        df = df.dropna(subset=['label']) # Drop rows where label mapping failed (NaN)
        df['label'] = df['label'].astype(int)
        dropped_bad_label = initial_count_before_label_drop - len(df)

        if df.empty:
            logging.warning(f"No valid data loaded from {filepath} after cleaning.")
            return None

        logging.info(f"Loaded {len(df)} samples from {filepath} (dropped {dropped_nan_tweet} for NaN tweet, {dropped_bad_label} for invalid label).")
        return df
    except Exception as e:
        logging.error(f"Error loading {filepath}: {e}", exc_info=True)
        return None

# --- 3. Custom PyTorch Dataset for Graph-RAG ---
class GraphRagNewsDataset(Dataset):
    """
    PyTorch Dataset for Graph-RAG. Retrieves context using a retriever function
    and tokenizes the combined text. Includes detailed error handling for debugging.
    """
    def __init__(self, texts, labels, tokenizer, max_len, retriever_func, num_retrieved_total):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len
        self.retriever_func = retriever_func
        self.num_retrieved_total = num_retrieved_total # Note: this isn't directly used in __getitem__ but passed for consistency

        # Ensure inputs are lists
        if not isinstance(self.texts, list): self.texts = list(self.texts)
        if not isinstance(self.labels, list): self.labels = list(self.labels)
        if len(self.texts) != len(self.labels):
            raise ValueError(f"Number of texts ({len(self.texts)}) and labels ({len(self.labels)}) must match.")
        logging.info(f"Dataset initialized with {len(self.texts)} samples.")

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        """
        Retrieves item, finds context, tokenizes, returns dict.
        Includes detailed logging for debugging worker errors.
        If an error occurs, it logs extensively and raises it if num_workers=0,
        or returns None if num_workers > 0 (which might still cause DataLoader issues).
        """
        # Log the start of processing for this index (use debug level to avoid excessive logs)
        # logging.debug(f"Processing index: {idx}")

        original_text = None # Initialize to None
        label = None
        try:
            # Check index bounds rigorously
            if not isinstance(idx, int) or not (0 <= idx < len(self.texts)):
                logging.error(f"Invalid index type or value: {idx} (type: {type(idx)}). Dataset size: {len(self.texts)}.")
                # Raise error immediately if in main process (num_workers=0)
                if not mp.current_process().daemon:
                     raise IndexError(f"Invalid index: {idx}")
                return None # Or handle differently if needed for worker processes

            original_text = str(self.texts[idx]) # Ensure text is string
            label = int(self.labels[idx]) # Ensure label is int

        except Exception as e:
            logging.error(f"Failed to get text/label at index {idx}. Error: {e}", exc_info=True)
            # If text/label access fails, we can't proceed with this item.
            if not mp.current_process().daemon: raise e # Re-raise if in main process
            return None # Returning None might cause collate_fn issues, but avoids crashing worker immediately

        # --- Retrieval Step ---
        retrieved_docs = []
        try:
            # logging.debug(f"Retrieving docs for index {idx}, text: '{original_text[:50]}...'")
            retrieved_docs = self.retriever_func(original_text)
            # Basic validation of retriever output
            if not isinstance(retrieved_docs, list):
                logging.warning(f"Retriever function did not return a list for index {idx}. Query: '{original_text[:50]}...'. Got: {type(retrieved_docs)}. Using empty list.")
                retrieved_docs = []
        except Exception as e:
            # Catch errors specifically from the retriever function
            logging.error(f"Failed during RETRIEVAL for index {idx}. Query: '{original_text[:100]}...'. Error: {e}", exc_info=True)
            retrieved_docs = [] # Fallback to empty context on retrieval failure
            if not mp.current_process().daemon: raise e # Re-raise if in main process

        # --- Tokenization Step ---
        combined_text = original_text
        try:
            # Ensure retrieved docs are strings and join them
            context = " ".join([str(doc) for doc in retrieved_docs if doc]) # Filter out None or empty docs
            sep_token = self.tokenizer.sep_token if self.tokenizer.sep_token else "[SEP]" # Use tokenizer's sep token

            # Combine original text and context if context exists
            if context:
                combined_text = f"{original_text} {sep_token} {context}"

            # logging.debug(f"Tokenizing combined text for index {idx}: '{combined_text[:100]}...'")
            encoding = self.tokenizer.encode_plus(
                combined_text,
                add_special_tokens=True,        # Add [CLS] and [SEP]
                max_length=self.max_len,        # Pad & truncate
                padding='max_length',           # Pad to max_length
                truncation=True,                # Truncate to max_length
                return_attention_mask=True,     # Return attention mask
                return_tensors='pt',            # Return PyTorch tensors
            )
        except Exception as e:
            # Catch errors specifically during tokenization
            logging.error(f"Failed during TOKENIZATION for index {idx}. Combined Text (start): '{combined_text[:100]}...'. Error: {e}", exc_info=True)
            if not mp.current_process().daemon: raise e # Re-raise if in main process
            return None # Tokenization failure means we can't return valid tensors

        # --- Return Result ---
        try:
            # Ensure tensors are correctly shaped (flatten removes the batch dimension added by return_tensors='pt')
            # and label is a tensor.
            return {
                'input_ids': encoding['input_ids'].flatten(),
                'attention_mask': encoding['attention_mask'].flatten(),
                'labels': torch.tensor(label, dtype=torch.long) # Ensure label is a tensor
            }
        except Exception as e:
            # Catch errors during the final dictionary creation/tensor conversion
            logging.error(f"Failed during final DICT CREATION for index {idx}. Label: {label}. Error: {e}", exc_info=True)
            if not mp.current_process().daemon: raise e # Re-raise if in main process
            return None


# --- 4. Evaluation Function ---
def evaluate_model(model, dataloader, device, loss_fn):
    """Evaluates the model on a given dataloader."""
    model.eval() # Set model to evaluation mode
    total_loss = 0
    all_preds = []
    all_labels = []

    # Handle cases where dataloader might be None or empty
    if dataloader is None or len(dataloader) == 0:
        logging.warning("Evaluation dataloader is empty or None. Skipping evaluation.")
        return 0, 0, 0, 0, 0 # Return zero metrics

    with torch.no_grad(): # Disable gradient calculations
        for batch in tqdm(dataloader, desc="Evaluating", leave=False):
            # Basic check for valid batch structure
            if not batch or 'input_ids' not in batch or 'attention_mask' not in batch or 'labels' not in batch:
                logging.warning("Skipping invalid batch during evaluation (missing keys or None).")
                continue
            # Check if tensors are empty (can happen if __getitem__ returned None and collate didn't handle it)
            if batch['input_ids'].numel() == 0:
                 logging.warning("Skipping empty batch during evaluation.")
                 continue

            try:
                # Move batch to device
                input_ids = batch['input_ids'].to(device)
                attention_mask = batch['attention_mask'].to(device)
                labels = batch['labels'].to(device)

                # Forward pass
                outputs = model(input_ids=input_ids, attention_mask=attention_mask)
                logits = outputs.logits

                # Calculate loss if loss_fn is provided
                if callable(loss_fn):
                    loss = loss_fn(logits, labels)
                    total_loss += loss.item()

                # Get predictions
                preds = torch.argmax(logits, dim=1)

                # Store predictions and labels for metrics calculation
                all_preds.extend(preds.cpu().numpy())
                all_labels.extend(labels.cpu().numpy())

            except Exception as e:
                logging.error(f"Error during evaluation batch processing: {e}", exc_info=True)
                # Decide whether to continue or stop evaluation on error
                continue # Skip the problematic batch

    # Check if any predictions/labels were collected
    if not all_labels or not all_preds:
        logging.warning("Evaluation resulted in no valid labels or predictions. Cannot compute metrics.")
        return 0, 0, 0, 0, 0

    # Calculate average loss
    num_batches = len(dataloader)
    avg_loss = total_loss / num_batches if callable(loss_fn) and num_batches > 0 else 0

    # Calculate metrics
    try:
        accuracy = accuracy_score(all_labels, all_preds)
        # Use zero_division=0 to avoid warnings/errors when a class has no predictions/labels
        precision, recall, f1, _ = precision_recall_fscore_support(
            all_labels, all_preds, average='binary', zero_division=0
        )
    except Exception as e:
        logging.error(f"Error calculating evaluation metrics: {e}", exc_info=True)
        accuracy, precision, recall, f1 = 0, 0, 0, 0 # Default to zero on error

    logging.info(f"Evaluation Results: Loss={avg_loss:.4f}, Acc={accuracy:.4f}, Prec={precision:.4f}, Rec={recall:.4f}, F1={f1:.4f}")
    return avg_loss, accuracy, precision, recall, f1


# --- Main Execution Block ---
if __name__ == "__main__":

    # --- Multiprocessing Setup ---
    # Set start method to 'spawn' BEFORE any multiprocessing-related imports or operations
    # This is crucial for CUDA compatibility with multiprocessing.
    try:
        current_start_method = mp.get_start_method(allow_none=True)
        # Only set if not already 'spawn' or if not set at all
        if current_start_method is None or current_start_method != 'spawn':
             # Check if setting the start method is necessary/possible
             if hasattr(mp, 'set_start_method'):
                mp.set_start_method('spawn', force=True)
                logging.info(f"Multiprocessing start method set to 'spawn' (was {current_start_method}).")
             else:
                 logging.warning("mp.set_start_method not available. Using default method.")
        else:
            logging.info("Multiprocessing start method already 'spawn'.")
    except (RuntimeError, ValueError) as e:
        # Log warning if setting fails, but don't necessarily exit
        logging.warning(f"Could not force multiprocessing start method to 'spawn': {e}. Using default: {mp.get_start_method(allow_none=True)}")

    # --- Seed and Device Setup ---
    np.random.seed(Config.seed)
    torch.manual_seed(Config.seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(Config.seed)
        logging.info(f"CUDA available. Using device: {Config.device}")
    else:
        logging.info(f"CUDA not available. Using device: {Config.device}")

    # --- Retriever Setup ---
    logging.info("--- Setting up Retriever ---")
    # Initialize variables to None
    train_df = None; train_texts = []; train_embeddings_cpu = None
    retriever_model = None; faiss_index = None
    try:
        # Load training data specifically for the retriever index
        train_df = load_data(Config.train_file)
        if train_df is None or train_df.empty:
            # This is critical, retriever cannot be built without data
            raise ValueError("Failed to load training data for retriever. Cannot proceed.")

        train_texts = train_df['tweet'].tolist() # Get the texts to be indexed
        logging.info(f"Loading retriever model: {Config.retriever_model_name}...")
        # Load the Sentence Transformer model
        retriever_model = SentenceTransformer(Config.retriever_model_name, device=Config.device)
        logging.info("Retriever model loaded.")

        # --- Embedding ---
        logging.info(f"Embedding {len(train_texts)} training documents for FAISS index...")
        batch_size_embed = 128 # Adjust based on GPU memory
        # Ensure model is on the correct device before encoding
        retriever_model.to(Config.device)
        # Encode the training texts
        train_embeddings = retriever_model.encode(
            train_texts,
            batch_size=batch_size_embed,
            convert_to_tensor=True,
            show_progress_bar=True, # Show progress bar during encoding
            device=Config.device
        )
        logging.info(f"Embeddings generated with shape: {train_embeddings.shape}")

        # Move embeddings to CPU for FAISS (FAISS typically uses CPU or requires specific GPU index)
        # Convert to float32 numpy array, which FAISS expects
        train_embeddings_cpu = train_embeddings.cpu().numpy().astype(np.float32)

        # Clean up GPU memory
        del train_embeddings; gc.collect()
        if Config.device == torch.device("cuda"): torch.cuda.empty_cache()

        # --- FAISS Indexing ---
        embedding_dim = train_embeddings_cpu.shape[1] # Get the dimension of embeddings
        logging.info(f"Building FAISS index (IndexFlatL2) with dimension={embedding_dim}...")
        # Using IndexFlatL2 for exact search (suitable for moderate dataset sizes)
        faiss_index = faiss.IndexFlatL2(embedding_dim)
        # Add the embeddings to the index
        faiss_index.add(train_embeddings_cpu)
        logging.info(f"FAISS index built successfully. Total vectors: {faiss_index.ntotal}")

    except Exception as e:
        # If retriever setup fails, log fatal error and exit
        logging.error(f"FATAL: Error during retriever setup: {e}", exc_info=True); exit(1)

    # --- Graph-like Retrieval Function Definition ---
    # This function uses the globally defined retriever_model, faiss_index, train_embeddings_cpu, train_texts
    def retrieve_documents_graph_like(query_text):
        """
        Retrieves documents using similarity-based graph neighborhood approach.
        1. Finds k primary neighbors of the query.
        2. Finds m secondary neighbors for each primary neighbor.
        3. Combines unique neighbors and returns the top max_total documents.
        """
        # Check if retriever components are initialized
        if retriever_model is None or faiss_index is None or train_embeddings_cpu is None or not train_texts:
            logging.warning("Retriever components not initialized. Cannot retrieve.")
            return []

        # Get retrieval parameters from Config
        k_primary = Config.num_retrieved_docs_primary
        m_secondary = Config.num_retrieved_docs_secondary
        max_total = Config.max_total_retrieved_docs

        try:
            # 1. Embed the query text
            # Ensure model is on the correct device (might be redundant if already there, but safe)
            retriever_model.to(Config.device)
            query_embedding = retriever_model.encode([query_text], convert_to_tensor=True, device=Config.device)
            query_embedding_np = query_embedding.cpu().numpy().astype(np.float32)
            # Clean up query embedding tensor from GPU memory
            del query_embedding; gc.collect()
            if Config.device == torch.device("cuda"): torch.cuda.empty_cache()


            # 2. Find primary neighbors (Top k_primary similar documents to the query)
            # faiss_index.search returns distances and indices
            distances_p, indices_p = faiss_index.search(query_embedding_np, k_primary)
            # Use a set to store unique indices, discard -1 (invalid index)
            primary_indices = set(indices_p[0])
            primary_indices.discard(-1)
            if not primary_indices:
                logging.debug(f"No primary neighbors found for query: '{query_text[:50]}...'")
                return [] # Return empty list if no primary neighbors

            # 3. Find secondary neighbors (Neighbors of the primary neighbors)
            secondary_indices = set()
            if m_secondary > 0 and primary_indices:
                # Convert set to list for indexing numpy array
                primary_indices_list = list(primary_indices)
                try:
                    # Get the embeddings of the primary neighbors from the CPU numpy array
                    primary_neighbor_embeddings = train_embeddings_cpu[primary_indices_list]
                except IndexError as ie:
                    # Handle potential index errors if primary_indices_list contains invalid values
                    logging.error(f"IndexError retrieving primary embeddings. Indices: {primary_indices_list}, Max index: {len(train_embeddings_cpu)-1}. Error: {ie}")
                    primary_neighbor_embeddings = np.array([]) # Fallback to empty array

                # Proceed only if we successfully retrieved primary neighbor embeddings
                if primary_neighbor_embeddings.shape[0] > 0:
                    # 4. Search for neighbors of the primary neighbors
                    # We search for m_secondary + 1 because the neighbor itself might be included
                    distances_s, indices_s = faiss_index.search(primary_neighbor_embeddings, m_secondary + 1)

                    # Iterate through the results for each primary neighbor
                    for i, primary_idx in enumerate(primary_indices_list):
                        # Iterate through the neighbors found for this primary neighbor
                        for neighbor_idx in indices_s[i]:
                            # Add the neighbor index if it's valid (-1 means no neighbor found)
                            # and it's not the primary neighbor itself
                            if neighbor_idx != -1 and neighbor_idx != primary_idx:
                                secondary_indices.add(neighbor_idx)

            # 5. Combine primary and secondary neighbors, ensuring uniqueness
            # Use set union to combine unique indices
            combined_indices = primary_indices.union(secondary_indices)
            # Convert back to list and limit to max_total documents
            final_indices = list(combined_indices)[:max_total]

            # 6. Retrieve the actual texts corresponding to the final indices
            retrieved_texts = []
            for i in final_indices:
                # Check index validity before accessing train_texts
                if 0 <= i < len(train_texts):
                    retrieved_texts.append(train_texts[i])
                else:
                    # Log a warning if an invalid index somehow ended up here
                    logging.warning(f"Retrieved invalid index {i} during text lookup (max: {len(train_texts)-1}). Skipping.")

            return retrieved_texts

        except Exception as e:
            # Catch any unexpected error during the retrieval process
            logging.error(f"ERROR in retrieve_documents_graph_like for query '{query_text[:50]}...': {e}", exc_info=True)
            # Attempt to clean up GPU memory in case of CUDA errors
            gc.collect()
            if Config.device == torch.device("cuda"): torch.cuda.empty_cache()
            return [] # Return empty list on failure

    # --- Load Data & Create DataLoaders ---
    logging.info("--- Loading Data and Creating DataLoaders ---")
    # Initialize variables
    val_df, test_df = None, None
    train_dataset, val_dataset, test_dataset = None, None, None
    train_dataloader, val_dataloader, test_dataloader = None, None, None
    classifier_tokenizer = None

    try:
        # Load validation and test dataframes using the same load_data function
        val_df = load_data(Config.val_file)
        test_df = load_data(Config.test_file)

        # Log warnings if validation or test data loading failed or resulted in empty dataframes
        if val_df is None or val_df.empty: logging.warning("Validation data loading failed or resulted in empty dataframe. Validation will be skipped.")
        if test_df is None or test_df.empty: logging.warning("Test data loading failed or resulted in empty dataframe. Final testing will be skipped.")

        # Load the tokenizer for the classifier model
        logging.info(f"Loading classifier tokenizer: {Config.classifier_model_name}...")
        classifier_tokenizer = AutoTokenizer.from_pretrained(Config.classifier_model_name)
        logging.info("Classifier tokenizer loaded.")

        # --- Create Datasets ---
        # Datasets need the data (texts, labels), tokenizer, max_len, and the retriever function
        logging.info("Creating datasets...")
        # Train dataset (use the already loaded train_df)
        if train_df is not None and not train_df.empty:
            train_dataset = GraphRagNewsDataset(
                texts=train_df['tweet'].tolist(),
                labels=train_df['label'].tolist(),
                tokenizer=classifier_tokenizer,
                max_len=Config.max_seq_length,
                retriever_func=retrieve_documents_graph_like, # Pass the retrieval function
                num_retrieved_total=Config.max_total_retrieved_docs
            )
        else:
            # Training dataset is essential, raise error if it couldn't be created
             logging.error("Cannot create train_dataset as train_df is invalid or empty.")
             raise ValueError("Training data is required to create the training dataset.")

        # Validation dataset
        if val_df is not None and not val_df.empty:
            val_dataset = GraphRagNewsDataset(
                texts=val_df['tweet'].tolist(),
                labels=val_df['label'].tolist(),
                tokenizer=classifier_tokenizer,
                max_len=Config.max_seq_length,
                retriever_func=retrieve_documents_graph_like,
                num_retrieved_total=Config.max_total_retrieved_docs
            )

        # Test dataset
        if test_df is not None and not test_df.empty:
            test_dataset = GraphRagNewsDataset(
                texts=test_df['tweet'].tolist(),
                labels=test_df['label'].tolist(),
                tokenizer=classifier_tokenizer,
                max_len=Config.max_seq_length,
                retriever_func=retrieve_documents_graph_like,
                num_retrieved_total=Config.max_total_retrieved_docs
            )
        logging.info("Datasets created.")

        # --- Determine Number of Workers for DataLoaders ---
        num_workers = 0 # Default to 0
        if Config.DEBUG_DATALOADER:
            logging.warning("DEBUG MODE ACTIVE: Setting num_workers = 0 for DataLoader. Error tracebacks will be more detailed.")
            num_workers = 0 # Force 0 workers for debugging
        else:
            # Use multiple workers only if not debugging, on CUDA, and start method is 'spawn'
            if Config.device == torch.device("cuda"):
                current_start_method = mp.get_start_method(allow_none=True)
                if current_start_method == 'spawn':
                    cpu_count = os.cpu_count()
                    # Heuristic: Use a fraction of CPU cores, capped at a reasonable number (e.g., 4)
                    # Avoid using too many workers, which can sometimes cause overhead or instability.
                    num_workers = min(4, cpu_count // 2 if cpu_count else 1) if cpu_count else 0
                    logging.info(f"Using {num_workers} dataloader workers (CUDA + spawn method detected).")
                else:
                    logging.warning(f"CUDA available but multiprocessing start method is '{current_start_method}' (expected 'spawn'). Using 0 workers for safety.")
            else:
                # Not using CUDA, typically safer to use 0 workers
                logging.info("Not using CUDA. Setting num_workers = 0.")

        # Enable pin_memory only if using CUDA and workers
        pin_memory = (num_workers > 0 and Config.device == torch.device("cuda"))
        logging.info(f"DataLoader pin_memory set to: {pin_memory}")

        # --- Create DataLoaders ---
        # Train DataLoader: Shuffle data, drop last batch if incomplete (can sometimes help stability)
        if train_dataset:
            train_dataloader = DataLoader(
                train_dataset,
                batch_size=Config.batch_size,
                shuffle=True,
                num_workers=num_workers,
                pin_memory=pin_memory,
                drop_last=True # Drop the last incomplete batch
            )
        # Validation DataLoader: No shuffling, don't drop last batch
        if val_dataset:
            val_dataloader = DataLoader(
                val_dataset,
                batch_size=Config.batch_size,
                shuffle=False,
                num_workers=num_workers,
                pin_memory=pin_memory
            )
        # Test DataLoader: No shuffling, don't drop last batch
        if test_dataset:
            test_dataloader = DataLoader(
                test_dataset,
                batch_size=Config.batch_size,
                shuffle=False,
                num_workers=num_workers,
                pin_memory=pin_memory
            )
        logging.info("DataLoaders created.")

        # Log the number of batches for confirmation
        if train_dataloader: logging.info(f"  Train DataLoader: {len(train_dataloader)} batches.")
        else: logging.error("Train dataloader creation failed. Training cannot proceed."); exit(1) # Exit if train loader fails
        if val_dataloader: logging.info(f"  Validation DataLoader: {len(val_dataloader)} batches.")
        else: logging.warning("Validation dataloader not created (likely due to missing/empty validation data).")
        if test_dataloader: logging.info(f"  Test DataLoader: {len(test_dataloader)} batches.")
        else: logging.warning("Test dataloader not created (likely due to missing/empty test data).")

    except Exception as e:
        # Catch any error during data loading or DataLoader creation
        logging.error(f"FATAL: Error during DataLoader creation phase: {e}", exc_info=True); exit(1)

    # --- Model Definition ---
    logging.info("--- Initializing Classifier Model ---")
    classifier_model = None
    try:
        logging.info(f"Loading classifier model: {Config.classifier_model_name}...")
        # Load the pre-trained model for sequence classification with 2 labels (real/fake)
        classifier_model = AutoModelForSequenceClassification.from_pretrained(
            Config.classifier_model_name,
            num_labels=2 # Binary classification
        )
        # Move the model to the configured device (GPU or CPU)
        classifier_model.to(Config.device)
        logging.info("Classifier model loaded and moved to device.")
    except Exception as e:
        # If model loading fails, log fatal error and exit
        logging.error(f"FATAL: Failed to initialize classifier model: {e}", exc_info=True); exit(1)

    # --- Training Setup ---
    logging.info("--- Setting up Training Components ---")
    optimizer = None; scheduler = None; loss_fn = None
    # Setup optimizer, scheduler, and loss function only if training is possible
    if train_dataloader and len(train_dataloader) > 0:
        try:
            # AdamW optimizer is commonly used with Transformers
            optimizer = AdamW(classifier_model.parameters(), lr=Config.learning_rate, eps=1e-8)

            # Calculate total training steps for scheduler
            # Needed for learning rate scheduling (linear warmup and decay)
            num_update_steps_per_epoch = len(train_dataloader) // Config.gradient_accumulation_steps
            total_steps = num_update_steps_per_epoch * Config.epochs

            # Linear scheduler with warmup
            scheduler = get_linear_schedule_with_warmup(
                optimizer,
                num_warmup_steps=Config.warmup_steps,
                num_training_steps=total_steps
            )

            # Cross-entropy loss for classification
            loss_fn = nn.CrossEntropyLoss()

            logging.info(f"Optimizer, Scheduler, and Loss Function initialized.")
            logging.info(f"Total training steps (considering grad accum): {total_steps}")
            logging.info(f"Warmup steps: {Config.warmup_steps}")

        except Exception as e:
            # If setup fails, log fatal error and exit
            logging.error(f"FATAL: Error during training setup (optimizer, scheduler, loss): {e}", exc_info=True); exit(1)
    else:
        # If train_dataloader is invalid, training cannot proceed
        logging.error("FATAL: Training dataloader is invalid or empty. Cannot setup training components."); exit(1)

    # --- Training Loop ---
    logging.info("--- Starting Training ---")
    best_val_f1 = -1.0 # Initialize best validation F1 score
    global_step = 0 # Track the number of optimizer steps
    training_start_time = time.time()

    # Loop through each epoch
    for epoch in range(Config.epochs):
        logging.info(f"===== Epoch {epoch + 1}/{Config.epochs} =====")
        epoch_start_time = time.time()

        # Set model to training mode
        classifier_model.train()
        total_train_loss = 0
        processed_batches = 0 # Count batches successfully processed in the epoch

        # Reset gradients at the start of the epoch
        optimizer.zero_grad()

        # Use tqdm for progress bar over the training dataloader
        progress_bar = tqdm(train_dataloader, desc=f"Epoch {epoch + 1} Training", leave=False)
        for step, batch in enumerate(progress_bar):
            # Basic check for valid batch structure
            if not batch or 'input_ids' not in batch or 'attention_mask' not in batch or 'labels' not in batch:
                logging.warning(f"Skipping invalid batch at step {step} in epoch {epoch + 1} (missing keys or None).")
                continue
            if batch['input_ids'].numel() == 0:
                 logging.warning(f"Skipping empty batch at step {step} in epoch {epoch + 1}.")
                 continue

            try:
                # Move batch to device
                input_ids = batch['input_ids'].to(Config.device)
                attention_mask = batch['attention_mask'].to(Config.device)
                labels = batch['labels'].to(Config.device)

                # Forward pass: Model returns loss when labels are provided
                outputs = classifier_model(input_ids=input_ids,
                                           attention_mask=attention_mask,
                                           labels=labels)
                loss = outputs.loss # Extract loss from model output

                # --- Gradient Accumulation ---
                if Config.gradient_accumulation_steps > 1:
                    # Normalize loss if accumulating gradients
                    loss = loss / Config.gradient_accumulation_steps

                # Backward pass: compute gradients
                loss.backward()

                # Accumulate loss for logging (scale back up)
                # Use item() to get Python number from tensor and avoid memory leaks
                total_train_loss += loss.item() * Config.gradient_accumulation_steps
                processed_batches += 1 # Increment count of processed batches

                # --- Optimizer Step ---
                # Perform optimizer step only after accumulating gradients for the specified number of steps
                if (step + 1) % Config.gradient_accumulation_steps == 0 or (step + 1) == len(train_dataloader):
                    # Clip gradients to prevent exploding gradients
                    torch.nn.utils.clip_grad_norm_(classifier_model.parameters(), 1.0)

                    # Update model parameters
                    optimizer.step()
                    # Update learning rate schedule
                    scheduler.step()
                    # Reset gradients for the next accumulation cycle
                    optimizer.zero_grad()

                    global_step += 1 # Increment global step count after optimizer step

                    # Update progress bar postfix with current loss and learning rate
                    current_lr = scheduler.get_last_lr()[0] if scheduler else Config.learning_rate
                    # Use the scaled loss for postfix display
                    progress_bar.set_postfix({'loss': f"{loss.item() * Config.gradient_accumulation_steps:.4f}", 'lr': f"{current_lr:.2e}", 'step': global_step})


            except Exception as e:
                # Handle errors during training step
                logging.error(f"Error during training step {step} in epoch {epoch + 1}: {e}", exc_info=True)
                logging.warning("Attempting to skip problematic batch and continue training...")
                # Crucially, reset gradients if an error occurred mid-accumulation cycle
                optimizer.zero_grad()
                # Try to clean up memory
                del batch, input_ids, attention_mask, labels, outputs, loss
                gc.collect()
                if Config.device == torch.device("cuda"): torch.cuda.empty_cache()
                continue # Skip to the next batch

        # Close the progress bar for the epoch
        progress_bar.close()

        # Calculate average training loss for the epoch
        epoch_duration = time.time() - epoch_start_time
        avg_train_loss = total_train_loss / processed_batches if processed_batches > 0 else 0
        current_lr = scheduler.get_last_lr()[0] if scheduler else Config.learning_rate
        logging.info(f"Epoch {epoch + 1} completed in {epoch_duration:.2f}s.")
        logging.info(f"  Average Training Loss: {avg_train_loss:.4f}")
        logging.info(f"  Current Learning Rate: {current_lr:.2e}")


        # --- Validation Step ---
        # Perform validation only if a validation dataloader exists
        if val_dataloader and len(val_dataloader) > 0:
            logging.info("--- Evaluating on Validation Set ---")
            val_loss, val_acc, val_prec, val_rec, val_f1 = evaluate_model(
                classifier_model, val_dataloader, Config.device, loss_fn
            )

            # Check if validation F1 score improved
            if val_f1 > best_val_f1:
                logging.info(f"Validation F1 improved ({best_val_f1:.4f} --> {val_f1:.4f}). Saving model...")
                best_val_f1 = val_f1 # Update best F1 score

                # Save the model and tokenizer
                os.makedirs(Config.output_dir, exist_ok=True) # Ensure output directory exists
                classifier_model.save_pretrained(Config.output_dir)
                if classifier_tokenizer: classifier_tokenizer.save_pretrained(Config.output_dir)
                logging.info(f"Model saved to {Config.output_dir}")
            else:
                logging.info(f"Validation F1 did not improve ({val_f1:.4f}). Best F1: {best_val_f1:.4f}")
        else:
            # Log if validation is skipped
            logging.warning("Skipping validation: Validation dataloader not available or empty.")

        # Clean up memory at the end of the epoch
        gc.collect()
        if Config.device == torch.device("cuda"): torch.cuda.empty_cache()

    # --- End of Training Loop ---
    training_duration = time.time() - training_start_time
    logging.info(f"--- Training Finished --- (Total Duration: {training_duration:.2f} seconds)")
    logging.info(f"Best Validation F1 achieved: {best_val_f1:.4f}")


    # --- Final Evaluation on Test Set ---
    # Perform final evaluation only if a test dataloader exists
    if test_dataloader and len(test_dataloader) > 0:
        logging.info("--- Evaluating on Test Set using the Best Model ---")
        try:
            best_model_path = Config.output_dir
            # Check if the best model was actually saved (requires config, model weights, tokenizer files)
            model_file_path = os.path.join(best_model_path, "pytorch_model.bin") # or model.safetensors
            config_file_path = os.path.join(best_model_path, "config.json")
            tokenizer_config_path = os.path.join(best_model_path, "tokenizer_config.json")

            # Check for essential files before loading
            essential_files_exist = (
                os.path.exists(model_file_path) or os.path.exists(os.path.join(best_model_path, "model.safetensors"))
            ) and os.path.exists(config_file_path) and os.path.exists(tokenizer_config_path)


            if essential_files_exist:
                logging.info(f"Loading best model from {best_model_path} for final test evaluation...")
                # Load the saved best model and tokenizer
                best_model = AutoModelForSequenceClassification.from_pretrained(best_model_path)
                best_tokenizer = AutoTokenizer.from_pretrained(best_model_path)
                best_model.to(Config.device) # Move loaded model to device
                logging.info("Best model and tokenizer loaded successfully.")

                # --- Re-create Test DataLoader with Best Tokenizer ---
                # It's good practice to ensure the test set uses the exact same tokenizer instance
                # that corresponds to the loaded model, although usually it's the same as classifier_tokenizer.
                logging.info("Re-creating test dataset and dataloader with loaded best tokenizer...")
                if test_df is not None and not test_df.empty:
                    test_dataset_final = GraphRagNewsDataset(
                        texts=test_df['tweet'].tolist(),
                        labels=test_df['label'].tolist(),
                        tokenizer=best_tokenizer, # Use the tokenizer loaded from the saved model directory
                        max_len=Config.max_seq_length,
                        retriever_func=retrieve_documents_graph_like,
                        num_retrieved_total=Config.max_total_retrieved_docs
                    )

                    # Use 0 workers for final evaluation for consistency, unless explicitly needed and safe
                    num_workers_test = 0
                    pin_memory_test = False # No pin_memory if num_workers is 0
                    # Optional: Allow workers for testing if not debugging and setup is correct
                    # if not Config.DEBUG_DATALOADER and Config.device == torch.device("cuda") and mp.get_start_method(allow_none=True) == 'spawn':
                    #     cpu_count = os.cpu_count()
                    #     num_workers_test = min(4, cpu_count // 2 if cpu_count else 1)
                    #     pin_memory_test = (num_workers_test > 0)

                    test_dataloader_final = DataLoader(
                        test_dataset_final,
                        batch_size=Config.batch_size,
                        shuffle=False, # No shuffling for test set
                        num_workers=num_workers_test,
                        pin_memory=pin_memory_test
                    )
                    logging.info(f"Final test dataloader created with {len(test_dataloader_final)} batches (workers={num_workers_test}, pin_memory={pin_memory_test}).")

                    # --- Evaluate Best Model on Test Set ---
                    logging.info("Evaluating best model on the final test set...")
                    # Use the loss function defined during training setup if available
                    final_loss_fn = loss_fn if 'loss_fn' in locals() and loss_fn is not None else None
                    if final_loss_fn is None: logging.warning("Loss function not available for final test evaluation. Loss will be 0.")

                    evaluate_model(best_model, test_dataloader_final, Config.device, final_loss_fn)

                else:
                    # This case should ideally not happen if test_dataloader was created earlier
                    logging.error("Cannot perform final evaluation: Test data (test_df) is unavailable or empty.")

            # --- Fallback: Evaluate Final State Model ---
            elif classifier_model:
                # If the best model wasn't saved (e.g., validation F1 never improved, or saving failed)
                # evaluate the model in its final state after training.
                logging.warning("No saved best model found or essential files missing in output directory.")
                logging.warning("Evaluating using the model's final state after training...")
                final_loss_fn = loss_fn if 'loss_fn' in locals() and loss_fn is not None else None
                if final_loss_fn is None: logging.warning("Loss function not available for final test evaluation. Loss will be 0.")
                # Use the original test_dataloader created earlier
                evaluate_model(classifier_model, test_dataloader, Config.device, final_loss_fn)
            else:
                # If no model is available at all (shouldn't happen if training ran)
                logging.error("Skipping final evaluation: No model available (neither saved best nor final training state).")

        except Exception as e:
            # Catch any error during the final evaluation phase
            logging.error(f"Error during final test evaluation phase: {e}", exc_info=True)
    else:
        # Log if test evaluation is skipped because the dataloader wasn't created
        logging.warning("Skipping final evaluation: Test dataloader not available or empty.")

    logging.info("--- Script Finished ---")


2025-04-10 23:48:23,014 - INFO - [MainProcess] Multiprocessing start method set to 'spawn' (was None).
2025-04-10 23:48:23,017 - INFO - [MainProcess] CUDA available. Using device: cuda
2025-04-10 23:48:23,018 - INFO - [MainProcess] --- Setting up Retriever ---
2025-04-10 23:48:23,467 - INFO - [MainProcess] Loaded 6420 samples from ../Constraint_English_Train.xlsx (dropped 0 for NaN tweet, 0 for invalid label).
2025-04-10 23:48:23,468 - INFO - [MainProcess] Loading retriever model: all-MiniLM-L6-v2...
2025-04-10 23:48:23,470 - INFO - [MainProcess] Load pretrained SentenceTransformer: all-MiniLM-L6-v2
2025-04-10 23:48:26,284 - INFO - [MainProcess] Retriever model loaded.
2025-04-10 23:48:26,286 - INFO - [MainProcess] Embedding 6420 training documents for FAISS index...


Batches:   0%|          | 0/51 [00:00<?, ?it/s]

2025-04-10 23:48:28,890 - INFO - [MainProcess] Embeddings generated with shape: torch.Size([6420, 384])
2025-04-10 23:48:29,308 - INFO - [MainProcess] Building FAISS index (IndexFlatL2) with dimension=384...
2025-04-10 23:48:29,314 - INFO - [MainProcess] FAISS index built successfully. Total vectors: 6420
2025-04-10 23:48:29,315 - INFO - [MainProcess] --- Loading Data and Creating DataLoaders ---
2025-04-10 23:48:29,452 - INFO - [MainProcess] Loaded 2140 samples from ../Constraint_English_Val.xlsx (dropped 0 for NaN tweet, 0 for invalid label).
2025-04-10 23:48:29,589 - INFO - [MainProcess] Loaded 2140 samples from ../english_test_with_labels.xlsx (dropped 0 for NaN tweet, 0 for invalid label).
2025-04-10 23:48:29,590 - INFO - [MainProcess] Loading classifier tokenizer: bert-base-uncased...
2025-04-10 23:48:29,868 - INFO - [MainProcess] Classifier tokenizer loaded.
2025-04-10 23:48:29,870 - INFO - [MainProcess] Creating datasets...
2025-04-10 23:48:29,871 - INFO - [MainProcess] Dataset

Epoch 1 Training:   0%|          | 0/802 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]